# Dataset 

In [1]:
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage

data_folder = '../../data/cifar'
trdt = datasets.CIFAR10(
    root=data_folder,
    train=True,
    download=True,
    transform=ToTensor(),
)
tedt = datasets.CIFAR10(
    root=data_folder,
    train=False,
    download=True,
    transform=ToTensor(),
)

trdl = DataLoader(trdt, batch_size=8)
tedl = DataLoader(tedt, batch_size=8)

Files already downloaded and verified
Files already downloaded and verified


# Model 

In [3]:
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import pickle
import torch
import os

# Define Model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv0   = nn.Conv2d    (   3,  8,   kernel_size=(3, 3),     bias=False  , padding=1)   
        self.max0    = nn.MaxPool2d (            kernel_size=(2,2)                   )   
        self.conv1   = nn.Conv2d    (   8,  16,  kernel_size=(3, 3),     bias=False , padding=1 )   
        self.max1    = nn.MaxPool2d (            kernel_size=(2,2)                   )   
        self.flat    = nn.Flatten   (                                                )   
        self.linear0 = nn.Linear    (            1024,  512                           )
        self.linear1 = nn.Linear    (            512,  10                            )
        self.soft    = nn.Softmax   (            dim=1                               )
        
        self.debug=False
        

    def forward(self,x):
        x_conv0      = self.conv0     (   x              )  
        x_max0       = self.max0      (   x_conv0        )
        x_conv1      = self.conv1     (   x_max0         )  
        x_max1       = self.max1      (   x_conv1        )
        x_flat       = self.flat      (   x_max1         )
        x_linear0    = self.linear0   (   x_flat         )
        x_linear1    = self.linear1   (   x_linear0      )
        x_prob       = self.soft      (   x_linear1      )

            
        return x_prob

# Initialize model and print

model = SimpleNet()


# Setting 

In [14]:
# Hyper Parameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 100
# batch_size_train = 64
# batch_size_test = 1000
learning_rate = 0.005
momentum = 0
log_interval = 1
torch.backends.cudnn.enabled = False

In [15]:
loss_fn = nn.CrossEntropyLoss()

In [16]:
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

# Train 

In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [23]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
model.to(device)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trdl, model, loss_fn, optimizer)
    test(tedl, model, loss_fn)

Epoch 1
-------------------------------
loss: 2.053529  [    0/50000]
loss: 2.047989  [ 8000/50000]
loss: 2.134475  [16000/50000]
loss: 2.105870  [24000/50000]
loss: 2.321122  [32000/50000]
loss: 2.123402  [40000/50000]
loss: 2.203446  [48000/50000]
Test Error: 
 Accuracy: 36.1%, Avg loss: 2.094341 

Epoch 2
-------------------------------
loss: 2.085620  [    0/50000]
loss: 1.977221  [ 8000/50000]
loss: 2.092948  [16000/50000]
loss: 2.032979  [24000/50000]
loss: 2.220207  [32000/50000]
loss: 1.997679  [40000/50000]
loss: 2.209704  [48000/50000]
Test Error: 
 Accuracy: 39.0%, Avg loss: 2.066014 

Epoch 3
-------------------------------
loss: 2.082739  [    0/50000]
loss: 1.828635  [ 8000/50000]
loss: 2.104442  [16000/50000]
loss: 1.939387  [24000/50000]
loss: 2.173897  [32000/50000]
loss: 2.013025  [40000/50000]
loss: 2.218328  [48000/50000]
Test Error: 
 Accuracy: 42.9%, Avg loss: 2.032158 

Epoch 4
-------------------------------
loss: 2.072047  [    0/50000]
loss: 1.888565  [ 8000/5

# Save and load 

In [ ]:
model_path = './weights/cifar_torch_simple.pth'
torch.save(model.state_dict(), model_path)

In [ ]:
model = SimpleNet()
model.load_state_dict(torch.load(model_path))

In [ ]:
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage

data_folder = '../../data/cifar'

tedt = datasets.CIFAR10(
    root=data_folder,
    train=False,
    download=True,
    transform=ToTensor(),
)

tedl = DataLoader(tedt, batch_size=4)

trit = iter(trdl)

In [ ]:
x,y = next(trit)

In [ ]:
# draw samples
from torchvision.utils import make_grid
show(make_grid(x))

In [ ]:
# predict
x = x.double()
pre = model(x)

pre_arg = pre.argmax(1)

In [ ]:
print(f'GT: {y.tolist()} \nPre: {pre_arg.tolist()}')